# CMPE 258 Assignment 1

In [0]:
# Imports
import numpy as np

In [2]:
# Connect Google Drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Load Dataset

In [3]:
train =  np.genfromtxt('train.csv', delimiter=",", skip_header=1)
train.shape

(59999, 785)

In [4]:
test =  np.genfromtxt('test.csv', delimiter=",", skip_header=1)
test.shape

(9999, 785)

## Image Details

In [0]:
image_width = 28
image_height = 28
image_size = image_width * image_height # 784
labels = 10

train_imgs = np.asfarray(train[:, 1:]) * (0.99 / 255) + 0.01
test_imgs = np.asfarray(test[:, 1:]) * (0.99 / 255) + 0.01

train_labels = np.asfarray(train[:, :1])
test_labels = np.asfarray(test[:, :1])


In [0]:
lr = np.arange(10)

# transform labels into one hot representation
train_labels_one_hot = (lr==train_labels).astype(np.float)
test_labels_one_hot = (lr==test_labels).astype(np.float)

# we don't want zeroes and ones in the labels neither:
train_labels_one_hot[train_labels_one_hot==0] = 0.01
train_labels_one_hot[train_labels_one_hot==1] = 0.99
test_labels_one_hot[test_labels_one_hot==0] = 0.01
test_labels_one_hot[test_labels_one_hot==1] = 0.99

## Neural Network Classifying the data

In [0]:
# Source: https://www.python-course.eu/neural_network_mnist.php 
import numpy as np

@np.vectorize
def sigmoid(x):
    return 1 / (1 + np.e ** -x)
activation_function = sigmoid

from scipy.stats import truncnorm

def truncated_normal(mean=0, sd=1, low=0, upp=10):
    return truncnorm((low - mean) / sd, 
                     (upp - mean) / sd, 
                     loc=mean, 
                     scale=sd)


class NeuralNetwork:
    
    def __init__(self, 
                 no_of_in_nodes, 
                 no_of_out_nodes, 
                 no_of_hidden_nodes,
                 learning_rate):
        self.no_of_in_nodes = no_of_in_nodes
        self.no_of_out_nodes = no_of_out_nodes
        self.no_of_hidden_nodes = no_of_hidden_nodes
        self.learning_rate = learning_rate 
        self.create_weight_matrices()
        
    def create_weight_matrices(self):
        """ 
        A method to initialize the weight 
        matrices of the neural network
        """
        rad = 1 / np.sqrt(self.no_of_in_nodes)
        X = truncated_normal(mean=0, 
                             sd=1, 
                             low=-rad, 
                             upp=rad)
        self.wih = X.rvs((self.no_of_hidden_nodes, 
                                       self.no_of_in_nodes))
        rad = 1 / np.sqrt(self.no_of_hidden_nodes)
        X = truncated_normal(mean=0, sd=1, low=-rad, upp=rad)
        self.who = X.rvs((self.no_of_out_nodes, 
                                         self.no_of_hidden_nodes))
        
    
    def train(self, input_vector, target_vector):
        """
        input_vector and target_vector can 
        be tuple, list or ndarray
        """
        
        input_vector = np.array(input_vector, ndmin=2).T
        target_vector = np.array(target_vector, ndmin=2).T
        
        output_vector1 = np.dot(self.wih, 
                                input_vector)
        output_hidden = activation_function(output_vector1)
        
        output_vector2 = np.dot(self.who, 
                                output_hidden)
        output_network = activation_function(output_vector2)
        
        output_errors = target_vector - output_network
        # update the weights:
        tmp = output_errors * output_network \
              * (1.0 - output_network)     
        tmp = self.learning_rate  * np.dot(tmp, 
                                           output_hidden.T)
        self.who += tmp


        # calculate hidden errors:
        hidden_errors = np.dot(self.who.T, 
                               output_errors)
        # update the weights:
        tmp = hidden_errors * output_hidden * \
              (1.0 - output_hidden)
        self.wih += self.learning_rate \
                          * np.dot(tmp, input_vector.T)
        

        
    
    def run(self, input_vector):
        # input_vector can be tuple, list or ndarray
        input_vector = np.array(input_vector, ndmin=2).T

        output_vector = np.dot(self.wih, 
                               input_vector)
        output_vector = activation_function(output_vector)
        
        output_vector = np.dot(self.who, 
                               output_vector)
        output_vector = activation_function(output_vector)
    
        return output_vector
            
    def confusion_matrix(self, data_array, labels):
        cm = np.zeros((10, 10), int)
        for i in range(len(data_array)):
            res = self.run(data_array[i])
            res_max = res.argmax()
            target = labels[i][0]
            cm[res_max, int(target)] += 1
        return cm    

    def precision(self, label, confusion_matrix):
        col = confusion_matrix[:, label]
        return confusion_matrix[label, label] / col.sum()
    
    def recall(self, label, confusion_matrix):
        row = confusion_matrix[label, :]
        return confusion_matrix[label, label] / row.sum()
        
    
    def evaluate(self, data, labels):
        corrects, wrongs = 0, 0
        for i in range(len(data)):
            res = self.run(data[i])
            res_max = res.argmax()
            if res_max == labels[i]:
                corrects += 1
            else:
                wrongs += 1
        return corrects, wrongs

In [8]:

epochs = 10

NN = NeuralNetwork(no_of_in_nodes = image_size, no_of_out_nodes = 10, no_of_hidden_nodes = 100, learning_rate = 0.1)

for epoch in range(epochs):  
    print("epoch: ", epoch)
    for i in range(len(train_imgs)):
        NN.train(train_imgs[i], 
                 train_labels_one_hot[i])
  
    corrects, wrongs = NN.evaluate(train_imgs, train_labels)
    print("accuracy train: ", corrects / ( corrects + wrongs))
    corrects, wrongs = NN.evaluate(test_imgs, test_labels)
    print("accuracy: test", corrects / ( corrects + wrongs))

epoch:  0
accuracy train:  0.9495491591526526
accuracy: test 0.9482948294829483
epoch:  1
accuracy train:  0.9659160986016434
accuracy: test 0.9613961396139614
epoch:  2
accuracy train:  0.9699994999916666
accuracy: test 0.961996199619962
epoch:  3
accuracy train:  0.9717995299921666
accuracy: test 0.9638963896389638
epoch:  4
accuracy train:  0.9771662861047684
accuracy: test 0.9658965896589659
epoch:  5
accuracy train:  0.9756829280488009
accuracy: test 0.963996399639964
epoch:  6
accuracy train:  0.9778329638827313
accuracy: test 0.9652965296529653
epoch:  7
accuracy train:  0.9781829697161619
accuracy: test 0.9665966596659666
epoch:  8
accuracy train:  0.9778662977716295
accuracy: test 0.9648964896489649
epoch:  9
accuracy train:  0.978632977216287
accuracy: test 0.965996599659966


In [9]:
'''
# Source : https://docs.google.com/presentation/d/1cYzq7TEGXRAhKF9P0eOI_q01kQAOajl-eS9h3CTGRLg/edit#slide=id.g5faeddb8e7_0_977
np.random.seed(1)
def relu(x):
  return (x >= 0) * x

def relu2deriv(output):
  return output >= 0

batch_size = 100
allpha, iterations = (0.001, 300)
pixels_per_image, num_labels, hidden_size = (784, 10, 100)

#weights_0_1 = 0.2*np.random((pixels_per_image, hidden_size)) - 0.1
#weights_1_2 = 0.2*np.random((hidden_size, num_labels)) - 0.1

weights_0_1 = 0
weights_1_2 = 0
'''

'\n# Source : https://docs.google.com/presentation/d/1cYzq7TEGXRAhKF9P0eOI_q01kQAOajl-eS9h3CTGRLg/edit#slide=id.g5faeddb8e7_0_977\nnp.random.seed(1)\ndef relu(x):\n  return (x >= 0) * x\n\ndef relu2deriv(output):\n  return output >= 0\n\nbatch_size = 100\nallpha, iterations = (0.001, 300)\npixels_per_image, num_labels, hidden_size = (784, 10, 100)\n\n#weights_0_1 = 0.2*np.random((pixels_per_image, hidden_size)) - 0.1\n#weights_1_2 = 0.2*np.random((hidden_size, num_labels)) - 0.1\n\nweights_0_1 = 0\nweights_1_2 = 0\n'

In [10]:
'''
# Source: https://docs.google.com/presentation/d/1cYzq7TEGXRAhKF9P0eOI_q01kQAOajl-eS9h3CTGRLg/edit#slide=id.g5faeddb8e7_0_977
for j in range(iterations):
  error, correct_cnt = (0.0, 0)
  for i in range(int(len(train) / batch_size)):
    batch_start, batch_end = ((i * batch_size), ((i+1) * batch_size))

    layer_0 = train[batch_start: batch_end]
    layer_1 = relu(np.dot(layer_0, weights_0_1))
    droupout_mask = np.random.randint(2, size=layer_1.shape)
    layer_1 *= droupout_mask * 2
    layer_2 = np.dot(layer_1, weights_1_2)

    error += np.sum((labels[batch_start:batch_end] - layer_2) ** 2)
    for k in range(batch_size):
      correct_cnt += int(np.argmax(layer_2[k:k+1]) == np.argmax(labels[batch_start+k:batch_start+k+1]))
      layer_2_delta = (labels[batch_start:batch_end] - layer_2) / batch_size
      layer_1_delta = layer_2_delta.dot(weights_1_2.T)* relu2deriv(layer_1)
      layer_1_delta *= droupout_mask

      weights_1_2 += alpha * layer_1.T.dot(layer_2_delta)
      weights_0_1 += alpha * layer_0.T.dot(layer_1_delta)

  if(j%10 == 0):
    test_error = 0.0
    test_correct_cnt = 0

    for i in range(len(test_image)):
      layer_0 = test_images[i:i+1]
      layer_1 = relu(np.dot(layer_0, weights_0_1))
      layer_2 = np.dot(layer_1, weights_1_2)
'''

'\n# Source: https://docs.google.com/presentation/d/1cYzq7TEGXRAhKF9P0eOI_q01kQAOajl-eS9h3CTGRLg/edit#slide=id.g5faeddb8e7_0_977\nfor j in range(iterations):\n  error, correct_cnt = (0.0, 0)\n  for i in range(int(len(train) / batch_size)):\n    batch_start, batch_end = ((i * batch_size), ((i+1) * batch_size))\n\n    layer_0 = train[batch_start: batch_end]\n    layer_1 = relu(np.dot(layer_0, weights_0_1))\n    droupout_mask = np.random.randint(2, size=layer_1.shape)\n    layer_1 *= droupout_mask * 2\n    layer_2 = np.dot(layer_1, weights_1_2)\n\n    error += np.sum((labels[batch_start:batch_end] - layer_2) ** 2)\n    for k in range(batch_size):\n      correct_cnt += int(np.argmax(layer_2[k:k+1]) == np.argmax(labels[batch_start+k:batch_start+k+1]))\n      layer_2_delta = (labels[batch_start:batch_end] - layer_2) / batch_size\n      layer_1_delta = layer_2_delta.dot(weights_1_2.T)* relu2deriv(layer_1)\n      layer_1_delta *= droupout_mask\n\n      weights_1_2 += alpha * layer_1.T.dot(laye

## Hidden Layers (Incomplete)

In [0]:
''' 
# Source: https://www.python-course.eu/neural_network_mnist.php 
import numpy as np

@np.vectorize
def sigmoid(x):
    return 1 / (1 + np.e ** -x)
activation_function = sigmoid

from scipy.stats import truncnorm

def truncated_normal(mean=0, sd=1, low=0, upp=10):
    return truncnorm((low - mean) / sd, 
                     (upp - mean) / sd, 
                     loc=mean, 
                     scale=sd)


class NeuralNetwork:
    
    def __init__(self, 
                 no_of_in_nodes, 
                 no_of_out_nodes, 
                 no_of_hidden_nodes,
                 learning_rate):
        self.no_of_in_nodes = no_of_in_nodes
        self.no_of_out_nodes = no_of_out_nodes
        self.no_of_hidden_nodes = no_of_hidden_nodes
        self.learning_rate = learning_rate 
        self.create_weight_matrices()
        
    def create_weight_matrices(self):
        """ 
        A method to initialize the weight 
        matrices of the neural network
        """
        rad = 1 / np.sqrt(self.no_of_in_nodes)
        X = truncated_normal(mean=0, 
                             sd=1, 
                             low=-rad, 
                             upp=rad)
        self.wih = X.rvs((self.no_of_hidden_nodes, 
                                       self.no_of_in_nodes))
        rad = 1 / np.sqrt(self.no_of_hidden_nodes)
        X = truncated_normal(mean=0, sd=1, low=-rad, upp=rad)
        self.who = X.rvs((self.no_of_out_nodes, 
                                         self.no_of_hidden_nodes))
        
    
    def train(self, input_vector, target_vector):
        """
        input_vector and target_vector can 
        be tuple, list or ndarray
        """
        
        input_vector = np.array(input_vector, ndmin=2).T
        target_vector = np.array(target_vector, ndmin=2).T
        
        output_vector1 = np.dot(self.wih, 
                                input_vector)
        output_hidden = activation_function(output_vector1)
        
        output_vector2 = np.dot(self.who, 
                                output_hidden)
        output_network = activation_function(output_vector2)
        
        output_errors = target_vector - output_network
        # update the weights:
        tmp = output_errors * output_network \
              * (1.0 - output_network)     
        tmp = self.learning_rate  * np.dot(tmp, 
                                           output_hidden.T)
        self.who += tmp


        # calculate hidden errors:
        hidden_errors = np.dot(self.who.T, 
                               output_errors)
        # update the weights:
        tmp = hidden_errors * output_hidden * \
              (1.0 - output_hidden)
        self.wih += self.learning_rate \
                          * np.dot(tmp, input_vector.T)
        

        
    
    def run(self, input_vector):
        # input_vector can be tuple, list or ndarray
        input_vector = np.array(input_vector, ndmin=2).T

        output_vector = np.dot(self.wih, 
                               input_vector)
        output_vector = activation_function(output_vector)
        
        output_vector = np.dot(self.who, 
                               output_vector)
        output_vector = activation_function(output_vector)
    
        return output_vector
            
    def confusion_matrix(self, data_array, labels):
        cm = np.zeros((10, 10), int)
        for i in range(len(data_array)):
            res = self.run(data_array[i])
            res_max = res.argmax()
            target = labels[i][0]
            cm[res_max, int(target)] += 1
        return cm    

    def precision(self, label, confusion_matrix):
        col = confusion_matrix[:, label]
        return confusion_matrix[label, label] / col.sum()
    
    def recall(self, label, confusion_matrix):
        row = confusion_matrix[label, :]
        return confusion_matrix[label, label] / row.sum()
        
    
    def evaluate(self, data, labels):
        corrects, wrongs = 0, 0
        for i in range(len(data)):
            res = self.run(data[i])
            res_max = res.argmax()
            if res_max == labels[i]:
                corrects += 1
            else:
                wrongs += 1
        return corrects, wrongs
'''